In [ ]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from torch.nn import *
from torch.optim import *
from torchvision.models import *
from sklearn.model_selection import *
from sklearn.metrics import f1_score,accuracy_score,precision_score
import wandb
import nltk
from nltk.stem.porter import *
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import f1_score
from sklearn import svm
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import warnings
import os
warnings.filterwarnings("ignore")
PROJECT_NAME = "Natural-Language-Processing-with-Disaster-Tweets"
np.random.seed(55)
stemmer = PorterStemmer()
device = "cuda"
os.environ["CUDA_LAUNCH_BLOCKING"]="1"
print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices("GPU") else "NOT AVAILABLE")

In [ ]:
class Sklearn_Data_Loader:
    def __init__(
        self,
        data: pd.DataFrame = pd.read_csv("./train.csv"),
        test: pd.DataFrame = pd.read_csv("./test.csv"),
        all_words: list = [],
        tags: list = [],
    ):
        self.data = data
        self.data = self.data.sample(frac=1)
        self.test = test
        self.X = self.data["text"]
        self.y = self.data["target"]
        # self.y = np.array(self.y.tolist())
        # self.y = self.y.reshape(-1, 1)
        self.all_words = all_words

    def create(self, count_vectorizer=False):
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            self.X, self.y, test_size=0.0625, shuffle=True
        )
        if count_vectorizer:
            self.vectorizer = CountVectorizer(binary=1)
        else:
            self.vectorizer = TfidfVectorizer()
        self.X_train = self.vectorizer.fit_transform(self.X_train)
        self.X_test = self.vectorizer.transform(self.X_test)
        # self.X_train = np.array(self.X_train)
        # self.y_train = np.array(self.y_train)
        # self.X_test = np.array(self.X_test)
        # self.y_test = np.array(self.y_test)
        return self.X_train, self.X_test, self.y_train, self.y_test, self.vectorizer

    def create_test(self):
        return self.vectorizer.transform(self.test["text"])

    def create_submission(self, model):
        preds = model.predict(self.create_test())
        print(preds)
        ids = self.test["id"]
        submission = {"id": [], "target": []}
        for pred, id in tqdm(zip(preds, ids)):
            submission["id"].append(id)
            submission["target"].append(int(pred))
        submission = pd.DataFrame(submission)
        return submission


In [ ]:
class Sklearn_Modelling:
    def train(self, model, X_train, y_train, X_test, y_test, name):
#         wandb.init(project=PROJECT_NAME, name=name)
        model.fit(X_train, y_train)
        preds = model.predict(X_test)
#         preds_prob = model.predict_proba(X_test)
        labels = sorted(set(list(y_test)))
#         wandb.log(
#             {
#                 "Accuracy": accuracy_score(preds, y_test),
#                 "F1 Score": f1_score(preds, y_test),
#                 "Precisions": precision_score(preds, y_test),
#             }
#         )
        print(
            {
                "Accuracy": accuracy_score(preds, y_test),
                "F1 Score": f1_score(preds, y_test),
                "Precisions": precision_score(preds, y_test),
            }
        )
#         wandb.sklearn.plot_confusion_matrix(y_test, preds, labels)
        # wandb.sklearn.plot_classifier(model, X_train, X_test, y_train, y_test, preds, preds_prob, {0:0,1:1},
        #                                                  model_name=f'{model}')
#         wandb.sklearn.plot_roc(y_test, preds_prob, labels)
#         wandb.sklearn.plot_precision_recall(y_test, preds_prob, labels)
#         wandb.sklearn.plot_confusion_matrix(y_test, preds, labels)
#         wandb.finish()
        return model, preds

In [ ]:
sdl = Sklearn_Data_Loader()
X_train, X_test, y_train, y_test, vectorizer = sdl.create()
sm = Sklearn_Modelling()
model = svm.SVC()
model.fit(X_train, y_train)
model, preds = sm.train(model, X_train, y_train, X_test, y_test, "name")
submission = sdl.create_submission(model)
submission.to_csv("./Sklearn-2.csv", index=False)